In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [14]:
df_48ind

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,2.37,0.12,-99.99,-5.19,1.29,8.65,2.50,50.21,-0.48,8.08,...,7.70,1.92,-23.79,0.07,1.87,4.61,-0.54,2.89,-5.77,5.20
1926-08-01,2.23,2.68,-99.99,27.03,6.50,16.81,-0.76,42.98,-3.58,-2.51,...,-2.38,4.85,5.39,-0.75,-0.13,11.83,2.57,5.30,0.32,6.76
1926-09-01,-0.57,1.58,-99.99,4.02,1.26,8.33,6.42,-4.91,0.73,-0.51,...,-5.54,0.08,-7.87,0.25,-0.56,-1.75,0.72,-3.06,-4.81,-3.86
1926-10-01,-0.46,-3.68,-99.99,-3.31,1.06,-1.40,-5.09,5.37,-4.68,0.12,...,-5.08,-2.62,-15.38,-2.20,-4.11,-11.82,-4.28,-5.74,-0.94,-8.49
1926-11-01,6.75,6.26,-99.99,7.29,4.55,0.00,1.82,-6.40,-0.54,1.87,...,3.84,1.61,4.67,6.52,4.33,-2.97,3.58,2.21,5.13,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-10-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-11-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


In [4]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)

## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [18]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

#df_BtoM.replace(-99.99, np.nan, inplace = True)
#df_BtoM.replace(-999, np.nan, inplace = True)
#df_BtoM.dropna(inplace = True)

df_monthly = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

# Conversion de la colonne 'Date' en format datetime avec un format spécifié
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'], format='%Y')

# Ajustement de la Date au 1er juillet
#df_monthly['Date'] = df_monthly['Date'] + pd.offsets.MonthBegin(2)
df_monthly

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
2,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
3,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
4,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1172,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1173,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1174,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


In [19]:
df_monthly = df_monthly.drop('Date', axis=1)

t = df_monthly.iloc[5:].values

df_test = pd.DataFrame(data = df_monthly.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)

df_test

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,0.90,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1926-08-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,0.90,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1926-09-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,0.90,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1926-10-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,0.90,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1926-11-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,0.90,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-10-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-11-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [6]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [7]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized